<img src='https://github.com/jtobelem-simplon/prepa-dp100/blob/master/images/top.png?raw=true'>

# Configuration (à lancer avant tous les notebooks)

In [2]:
# version de python
import platform
platform.python_version()

'3.8.5'

In [10]:
# la liste des packages installés
!conda list

# packages in environment at /home/lab/anaconda3/envs/azure:
#
# Name                    Version                   Build  Channel
_libgcc_mutex             0.1                        main  
adal                      1.2.4                    pypi_0    pypi
applicationinsights       0.11.9                   pypi_0    pypi
argon2-cffi               20.1.0           py38h7b6447c_1  
attrs                     20.2.0                     py_0  
azure-common              1.1.25                   pypi_0    pypi
azure-core                1.8.1                    pypi_0    pypi
azure-graphrbac           0.61.1                   pypi_0    pypi
azure-identity            1.2.0                    pypi_0    pypi
azure-mgmt-authorization  0.61.0                   pypi_0    pypi
azure-mgmt-containerregistry 2.8.0                    pypi_0    pypi
azure-mgmt-keyvault       2.2.0                    pypi_0    pypi
azure-mgmt-resource       10.2.0                   pypi_0    pypi
azure-mgmt-storage        1

In [11]:
# version de la SDK azureml
import azureml.core
print("Ready to use Azure ML", azureml.core.VERSION)

Ready to use Azure ML 1.13.0


Si le notebook est executé en dehors d'Azure, il faut télécharger le fichier config.json depuis le portail https://portal.azure.com/, et le mettre dans le workspace qui contient le notebook.

Si le notebook est exécuté directement depuis le workspace Azure, le fichier de config devrait déjà être là.

In [3]:
# connexion au workspace
from azureml.core import Workspace

ws = Workspace.from_config()
print(ws.name, "loaded")

jt-dp100 loaded


In [13]:
from azureml.core import ComputeTarget, Datastore, Dataset

print("- Compute Targets :")
for compute_name in ws.compute_targets:
    compute = ws.compute_targets[compute_name]
    print("\t", compute.name, ':', compute.type)
    
print("- Datastores :")
for datastore_name in ws.datastores:
    datastore = Datastore.get(ws, datastore_name)
    print("\t", datastore.name, ':', datastore.datastore_type)
    
print("- Datasets :")
for dataset_name in list(ws.datasets.keys()):
    dataset = Dataset.get_by_name(ws, dataset_name)
    print("\t", dataset.name)

- Compute Targets :
	 vm-ds3-v2 : ComputeInstance
	 aml-cluster : AmlCompute
	 vm-ds3-v2-b : ComputeInstance
- Datastores :
	 azureml_globaldatasets : AzureBlob
	 workspacefilestore : AzureFile
	 workspaceblobstore : AzureBlob
- Datasets :
	 titanic test dataset
	 titanic train dataset
	 diabetes file dataset
	 diabetes dataset


# Enregistrer une source de donnée

Upload des fichiers csv, crée un dataset à partir de ces fichiers et enregistre la référence sur le working set.

In [5]:
from azureml.core import Dataset

dataset_name = 'titanic train dataset'
description='titanic training data'
file='./data/titanic/train.csv'
target_path='titanic-data/'

default_ds = ws.get_default_datastore()

if dataset_name not in ws.datasets:
    default_ds.upload_files(files=[file], target_path=target_path, overwrite=True, show_progress=True)

    #Create a tabular dataset from the path on the datastore (this may take a short while)
    tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, file))

    # Register the tabular dataset
    try:
        tab_data_set.register(workspace=ws, 
                                name=dataset_name,
                                description=description,
                                tags = {'format':'CSV'},
                                create_new_version=True)
        
        print('Dataset registered.')
    except Exception as ex:
        print(ex)
else:
    print('Dataset already registered.')

Dataset already registered.


On peut vérifier le contenu du dataset.

In [ ]:
titanic_ds = ws.datasets.get("titanic train dataset")
titanic_ds.to_pandas_dataframe().head()

# Gestion des environnements

https://docs.microsoft.com/fr-fr/azure/machine-learning/how-to-use-environments

https://docs.microsoft.com/fr-fr/azure/machine-learning/resource-curated-environments

In [ ]:
from azureml.core import Environment

envs = Environment.list(workspace=ws)

for env in envs:
    if env.startswith("AzureML-AutoML"):
        print("Nom :",env)
        print()
        print("Packages :", envs[env].python.conda_dependencies.serialize_to_string())

In [ ]:
from azureml.core import Workspace, Environment

ws = Workspace.from_config()
env = Environment.get(workspace=ws, name="AzureML-AutoML")

# Création d'une instance de calcul

## Création d'une aml compute instance

In [ ]:
from azureml.core.compute import ComputeTarget, ComputeInstance
from azureml.core.compute_target import ComputeTargetException

compute_name = "vm-ds3-v2"

# Verify that instance does not exist already
if compute_name in ws.compute_targets:
    print('Found existing instance, use it : ' + compute_name)
else:
    print('creating a new compute target...')
    compute_config = ComputeInstance.provisioning_configuration(
        vm_size='STANDARD_D3_V2',
        ssh_public_access=False
    )
    instance = ComputeInstance.create(ws, compute_name, compute_config)
    instance.wait_for_completion(show_output=True)

## Création d'un cluster

Il faut définir le nom du cluster, ainsi que les nombres de noeuds min et max.

In [ ]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
import os

# choose a name for your cluster
compute_name = os.environ.get("AML_COMPUTE_CLUSTER_NAME", "aml-cluster")
compute_min_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MIN_NODES", 0)
compute_max_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MAX_NODES", 4)

# This example uses CPU VM. For using GPU VM, set SKU to STANDARD_NC6
vm_size = os.environ.get("AML_COMPUTE_CLUSTER_SKU", "STANDARD_D2_V2")


if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print('found compute target. just use it : ' + compute_name)
else:
    print('creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size=vm_size,
                                                                min_nodes=compute_min_nodes,
                                                                max_nodes=compute_max_nodes)

    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)

    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

    print(compute_target.get_status().serialize())

## Pour utiliser une instance déjà créee

In [ ]:
compute_name = "aml-cluster"

if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    print('found compute target. just use it. ' + compute_name)
        
else :
    compute_target = 'local'
    print('compute target not found. work locally.')

# Ne pas oublier à la fin de l'expérience!!
(si votre travail à utilisé une instance de calcul)

<img src='https://github.com/jtobelem-simplon/prepa-dp100/blob/master/images/down.png?raw=true'>



In [ ]:
# stop toutes les instances de calcul
from azureml.core.compute import ComputeTarget, AmlCompute, ComputeInstance
from azureml.core.compute_target import ComputeTargetException

for compute in ComputeTarget.list(ws):
    if type(compute) is ComputeInstance and compute.get_status().state != 'Stopped':
        print('try to stop compute', compute.name)
        compute.stop(show_output=True)

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute, ComputeInstance

# liste tous les compute pour vérifier qu'elles sont éteintes
for compute in ComputeTarget.list(ws):
    if type(compute) is ComputeInstance:
        print(compute.name, compute.get_status())

# Ressources

[api azure](https://docs.microsoft.com/en-us/python/api/azureml-core)

[parcours d'apprentissage microsoft](https://docs.microsoft.com/fr-fr/learn/paths/build-ai-solutions-with-azure-ml-service/)

[le repository microsoft](https://github.com/MicrosoftDocs/mslearn-aml-labs.git)

from azureml.core import Environment

envs = Environment.list(workspace=ws)

for env in envs:
    if env.startswith("AzureML-AutoML"):
        print("Nom :",env)
        print()
        print("Packages :", envs[env].python.conda_dependencies.serialize_to_string())

In [6]:
titanic_ds = ws.datasets.get("titanic train dataset")
titanic_ds.to_pandas_dataframe().head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,None,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,None,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,None,S


# Gestion des environnements

https://docs.microsoft.com/fr-fr/azure/machine-learning/how-to-use-environments

https://docs.microsoft.com/fr-fr/azure/machine-learning/resource-curated-environments

In [ ]:
from azureml.core import Environment

envs = Environment.list(workspace=ws)

for env in envs:
    if env.startswith("AzureML-AutoML"):
        print("Nom :",env)
        print()
        print("Packages :", envs[env].python.conda_dependencies.serialize_to_string())

In [ ]:
from azureml.core import Workspace, Environment

ws = Workspace.from_config()
env = Environment.get(workspace=ws, name="AzureML-AutoML")

# Création d'une instance de calcul

## Création d'une aml compute instance

In [ ]:
from azureml.core.compute import ComputeTarget, ComputeInstance
from azureml.core.compute_target import ComputeTargetException

compute_name = "vm-ds3-v2"

# Verify that instance does not exist already
if compute_name in ws.compute_targets:
    print('Found existing instance, use it : ' + compute_name)
else:
    print('creating a new compute target...')
    compute_config = ComputeInstance.provisioning_configuration(
        vm_size='STANDARD_D3_V2',
        ssh_public_access=False
    )
    instance = ComputeInstance.create(ws, compute_name, compute_config)
    instance.wait_for_completion(show_output=True)

## Création d'un cluster

Il faut définir le nom du cluster, ainsi que les nombres de noeuds min et max.

In [ ]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
import os

# choose a name for your cluster
compute_name = os.environ.get("AML_COMPUTE_CLUSTER_NAME", "aml-cluster")
compute_min_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MIN_NODES", 0)
compute_max_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MAX_NODES", 4)

# This example uses CPU VM. For using GPU VM, set SKU to STANDARD_NC6
vm_size = os.environ.get("AML_COMPUTE_CLUSTER_SKU", "STANDARD_D2_V2")


if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print('found compute target. just use it : ' + compute_name)
else:
    print('creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size=vm_size,
                                                                min_nodes=compute_min_nodes,
                                                                max_nodes=compute_max_nodes)

    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)

    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

    print(compute_target.get_status().serialize())

## Pour utiliser une instance déjà créee

In [ ]:
compute_name = "aml-cluster"

if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    print('found compute target. just use it. ' + compute_name)
        
else :
    compute_target = 'local'
    print('compute target not found. work locally.')

# Ne pas oublier à la fin de l'expérience!!
(si votre travail à utilisé une instance de calcul)

<img src='https://github.com/jtobelem-simplon/prepa-dp100/blob/master/images/down.png?raw=true'>



In [ ]:
# stop toutes les instances de calcul
from azureml.core.compute import ComputeTarget, AmlCompute, ComputeInstance
from azureml.core.compute_target import ComputeTargetException

for compute in ComputeTarget.list(ws):
    if type(compute) is ComputeInstance and compute.get_status().state != 'Stopped':
        print('try to stop compute', compute.name)
        compute.stop(show_output=True)

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute, ComputeInstance

# liste tous les compute pour vérifier qu'elles sont éteintes
for compute in ComputeTarget.list(ws):
    if type(compute) is ComputeInstance:
        print(compute.name, compute.get_status())

# Ressources

[api azure](https://docs.microsoft.com/en-us/python/api/azureml-core)

[parcours d'apprentissage microsoft](https://docs.microsoft.com/fr-fr/learn/paths/build-ai-solutions-with-azure-ml-service/)

[le repository microsoft](https://github.com/MicrosoftDocs/mslearn-aml-labs.git)

In [15]:
from azureml.core import Environment

envs = Environment.list(workspace=ws)

for env in envs:
    if env.startswith("AzureML-AutoML"):
        print("Nom :",env)
        print()
        print("Packages :", envs[env].python.conda_dependencies.serialize_to_string())

Nom : AzureML-AutoML

Packages : channels:
- anaconda
- conda-forge
- pytorch
dependencies:
- python=3.6.2
- pip:
  - azureml-core==1.13.0
  - azureml-pipeline-core==1.13.0
  - azureml-telemetry==1.13.0
  - azureml-defaults==1.13.0
  - azureml-interpret==1.13.0
  - azureml-explain-model==1.13.0
  - azureml-automl-core==1.13.0
  - azureml-automl-runtime==1.13.0.post1
  - azureml-train-automl-client==1.13.0.post2
  - azureml-train-automl-runtime==1.13.0
  - inference-schema
  - py-cpuinfo==5.0.0
- numpy~=1.18.0
- scikit-learn==0.22.1
- pandas~=0.25.0
- py-xgboost<=0.90
- fbprophet==0.5
- holidays==0.9.11
- setuptools-git
- psutil>5.0.0,<6.0.0
name: azureml_ff9800a75438efccb779648369b8d727

Nom : AzureML-AutoML-DNN-Vision-GPU

Packages : dependencies:
- python=3.7
- pip:
  - azureml-core==1.13.0
  - azureml-dataset-runtime==1.13.0
  - azureml-contrib-dataset==1.13.0
  - azureml-telemetry==1.13.0
  - azureml-automl-core==1.13.0
  - azureml-automl-runtime==1.13.0.post1
  - azureml-train-aut

In [17]:
from azureml.core import Workspace, Environment

ws = Workspace.from_config()
env = Environment.get(workspace=ws, name="AzureML-AutoML")

# Création d'une instance de calcul

## Création d'une aml compute instance

In [ ]:
from azureml.core.compute import ComputeTarget, ComputeInstance
from azureml.core.compute_target import ComputeTargetException

compute_name = "vm-ds3-v2"

# Verify that instance does not exist already
if compute_name in ws.compute_targets:
    print('Found existing instance, use it : ' + compute_name)
else:
    print('creating a new compute target...')
    compute_config = ComputeInstance.provisioning_configuration(
        vm_size='STANDARD_D3_V2',
        ssh_public_access=False
    )
    instance = ComputeInstance.create(ws, compute_name, compute_config)
    instance.wait_for_completion(show_output=True)

## Création d'un cluster

Il faut définir le nom du cluster, ainsi que les nombres de noeuds min et max.

In [ ]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
import os

# choose a name for your cluster
compute_name = os.environ.get("AML_COMPUTE_CLUSTER_NAME", "aml-cluster")
compute_min_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MIN_NODES", 0)
compute_max_nodes = os.environ.get("AML_COMPUTE_CLUSTER_MAX_NODES", 4)

# This example uses CPU VM. For using GPU VM, set SKU to STANDARD_NC6
vm_size = os.environ.get("AML_COMPUTE_CLUSTER_SKU", "STANDARD_D2_V2")


if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    if compute_target and type(compute_target) is AmlCompute:
        print('found compute target. just use it : ' + compute_name)
else:
    print('creating a new compute target...')
    provisioning_config = AmlCompute.provisioning_configuration(vm_size=vm_size,
                                                                min_nodes=compute_min_nodes,
                                                                max_nodes=compute_max_nodes)

    compute_target = ComputeTarget.create(ws, compute_name, provisioning_config)

    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

    print(compute_target.get_status().serialize())

## Pour utiliser une instance déjà créee

In [ ]:
compute_name = "aml-cluster"

if compute_name in ws.compute_targets:
    compute_target = ws.compute_targets[compute_name]
    print('found compute target. just use it. ' + compute_name)
        
else :
    compute_target = 'local'
    print('compute target not found. work locally.')

# Ne pas oublier à la fin de l'expérience!!
(si votre travail à utilisé une instance de calcul)

<img src='https://github.com/jtobelem-simplon/prepa-dp100/blob/master/images/down.png?raw=true'>



In [ ]:
# stop toutes les instances de calcul
from azureml.core.compute import ComputeTarget, AmlCompute, ComputeInstance
from azureml.core.compute_target import ComputeTargetException

for compute in ComputeTarget.list(ws):
    if type(compute) is ComputeInstance and compute.get_status().state != 'Stopped':
        print('try to stop compute', compute.name)
        compute.stop(show_output=True)

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute, ComputeInstance

# liste tous les compute pour vérifier qu'elles sont éteintes
for compute in ComputeTarget.list(ws):
    if type(compute) is ComputeInstance:
        print(compute.name, compute.get_status())

# Ressources

[api azure](https://docs.microsoft.com/en-us/python/api/azureml-core)

[parcours d'apprentissage microsoft](https://docs.microsoft.com/fr-fr/learn/paths/build-ai-solutions-with-azure-ml-service/)

[le repository microsoft](https://github.com/MicrosoftDocs/mslearn-aml-labs.git)